In [ ]:
from endstate_correction.protocol import Results
import matplotlib.pyplot as plt
from typing import Tuple
import seaborn as sns
from pymbar import bar
from pymbar.other_estimators import exp
import numpy as np
import pickle
from itertools import zip_longest 
from endstate_correction.constant import zinc_systems, blacklist

def parse_energy_matrix(f):
    i,j = 0,0
    energy = np.zeros((60,60))
    for l in f.readlines():
        if 'Energy' in l:
            energy[i][j] = float(l.split(' ')[-1])
            j += 1
            
            if j == 60:
                j = 0
                i += 1
    return energy


# bootstrap metric
def bootstrap_exp(fct:callable, dE_values)-> Tuple[float, float, float]:
    """bootstrap exponential average estimator

    Args:
        fct (callable): function to bootstrap
        dE_values (list): energies to bootstrap

    Returns:
        Tuble: mean, lower, upper
    """
    
    assert callable(fct) == True
    bootstrapped_metric = []
    # bootstrap metric to generate test distribution
    for _ in range(1000):
        indices = np.random.choice(range(0, len(dE_values)), size=len(dE_values), replace=True)
        selection = np.take(dE_values, indices)
        r = fct(selection)['Delta_f']
        bootstrapped_metric.append(r)
    
    # define 90% CI
    alpha = 10.0
    lower_p = alpha / 2.0
    # get value at or near percentile (take a look at the definition of percentile if 
    # you have less than 100 values to make sure you understand what is happening)
    lower = np.percentile(bootstrapped_metric, lower_p)
    upper_p = (100 - alpha) + (alpha / 2.0)
    upper = np.percentile(bootstrapped_metric, upper_p)
    # calculate true mean
    mean = fct(dE_values)['Delta_f']

    return mean, lower, upper

def bootstrap_bar(fct:callable, fw_f:list, rv_f:list)-> Tuple[float, float, float]:
    """bootstrap BAR estimator

    Args:
        fct (callable): function to bootstrap
        fw_f (list): forward work values to bootstrap
        rv_f (list): reverse work values to bootstrap

    Returns:
        Tuble: mean, lower, upper
    """

    assert callable(fct) == True
    bootstrapped_metric = []
    # bootstrap metric to generate test distribution
    for _ in range(1000):
        indices_fw = np.random.choice(range(0, len(fw_f)), size=len(fw_f), replace=True)
        selection_fw = np.take(fw_f, indices_fw)
        indices_rv = np.random.choice(range(0, len(rv_f)), size=len(rv_f), replace=True)
        selection_rv = np.take(rv_f, indices_rv)
        r = fct(selection_fw, selection_rv)['Delta_f']
        bootstrapped_metric.append(r)
    
    # define 90% CI
    alpha = 10.0
    lower_p = alpha / 2.0
    # get value at or near percentile (take a look at the definition of percentile if 
    # you have less than 100 values to make sure you understand what is happening)
    lower = np.percentile(bootstrapped_metric, lower_p)
    upper_p = (100 - alpha) + (alpha / 2.0)
    upper = np.percentile(bootstrapped_metric, upper_p)
    # calculate true mean
    mean = fct(fw_f, rv_f)['Delta_f']

    return mean, lower, upper

def plot_dist(r:Results, system_name:str, axs:plt.Axes):
        
    """ plot results for a single system
    Args:
        r (Results): results object
        system_name (str): system name
        axs (plt.Axes): axes to plot on

    Returns:
        plt.Axes: axes with plot   
    """
            
    colors = sns.color_palette("flare", n_colors = 201) 
   
    # draw histogramms and results for FEP and NEQ
    ##################################################
    # start with FEP
    w_F, w_R = r.dE_mm_to_qml, r.dE_qml_to_mm
    rexp_f, rexp_f_lower, rexp_f_upper = bootstrap_exp(exp, w_F)
    rexp_r, rexp_r_lower, rexp_r_upper = bootstrap_exp(exp, w_R)
    bar_bi, bar_bi_lower, bar_bi_upper = bootstrap_bar(bar, w_F, w_R)
    
    # equ dG
    equ_rs = [equ.compute_free_energy_differences()["Delta_f"][0][-1] for equ in r.equ_mbar]

    # offset everything and change sign
    # calculate offset
    tmp = np.append(r.dE_mm_to_qml, r.dE_qml_to_mm * -1)
    min_F = min(tmp)

    rexp_f, rexp_f_lower, rexp_f_upper = rexp_f-min_F, rexp_f_lower-min_F, rexp_f_upper-min_F
    rexp_r, rexp_r_lower, rexp_r_upper = (rexp_r * -1)-min_F, (rexp_r_lower * -1)-min_F, (rexp_r_upper * -1)-min_F
    bar_bi, bar_bi_lower, bar_bi_upper = bar_bi-min_F, bar_bi_lower-min_F, bar_bi_upper-min_F
    equ_r = np.average(equ_rs)-min_F
    # 90% CI: https://sphweb.bumc.bu.edu/otlt/MPH-Modules/PH717-QuantCore/PH717-Module6-RandomError/PH717-Module6-RandomError11.html
    # CI = +- t-score(/sigma / sqrt(n))
    equ_dDG = 2.920 * (np.std(equ_rs) / np.sqrt(len(equ_rs))) 
    
    print(f'{np.round(equ_r, 2)} [{np.round(equ_dDG, 2)};{np.round(equ_dDG, 2)}],' ,end =" " )
    print(f'{np.round(rexp_f, 2)} [{np.round(rexp_f_lower, 2)};{np.round(rexp_f_upper, 2)}],',end =" " )
    print(f'{np.round(rexp_r, 2)} [{np.round(rexp_r_lower, 2)};{np.round(rexp_r_upper, 2)}],',end =" " )
    print(f'{np.round(bar_bi, 2)} [{np.round(bar_bi_lower, 2)};{np.round(bar_bi_upper, 2)}],',end =" ")

    axs[0].set_title(f'FEP - {system_name}', fontsize=15)
    ax1 = sns.kdeplot(w_F-min_F, label='f_forw', ax=axs[0], fill=True, alpha=.5,color=colors[0])
    y1, x = np.histogram(w_F-min_F, density=True)
    y2, x = np.histogram((w_R*-1)-min_F, density=True)

    y_max = max([y1.max(), y2.max()])
    #sns.rugplot(w_F-min_F, ax=ax1, lw=1, alpha=.1,color=colors[0])
    axs[0].scatter(w_F-min_F, np.random.uniform(low=(y_max/6) *  -1, high=0., size=len(w_F)), s=.4, color=colors[0], alpha=0.8)

    ax2 = sns.kdeplot((w_R*-1)-min_F, label='-f_rev', ax=axs[0], fill=True, alpha=.5,color=colors[-1])
    #sns.rugplot((w_R*-1)-min_F, ax=ax2, lw=1, alpha=.1,color=colors[-1])
    axs[0].scatter((w_R*-1)-min_F, np.random.uniform(low=(y_max/6) *  -1, high=0., size=len(w_R)), s=.4, color=colors[-1], alpha=0.8)

    # bidirectional estimate
    axs[0].axvline(x = bar_bi,
            color = 'red',lw=3 ,ls=':',
            label = r'$\Delta G_{bid}$')

    # EXP forward
    axs[0].axvline(x = rexp_f, 
            color = colors[0], lw=3,ls=':',
            label = r'$\Delta G_{forw}$', alpha=0.5)

    # EXP reverse
    axs[0].axvline(x = rexp_r, 
            color = colors[-1], lw=3,ls=':', alpha=0.5,
            label = r'$\Delta G_{rev}$')
    
    # equ dG
    axs[0].axvline(x=equ_r, 
            color = 'black',lw=3,ls='-', alpha=0.5,
            label = r'$\Delta G_{equ}$')


    textstr_fep = f'''$\Delta G_{{forw}}$ = {rexp_f:.2f} [{rexp_f_lower:.2f}; {rexp_f_upper:.2f}]
$\Delta G_{{rev}}$ = {rexp_r:.2f} [{rexp_r_upper:.2f}; {rexp_r_lower:.2f}]
$\Delta G_{{bid}}$ = {bar_bi:.2f} [{bar_bi_lower:.2f}; {bar_bi_upper:.2f}]
$\Delta G_{{equ}}$ = {equ_r:.2f} [{equ_r-equ_dDG:.2f}; {equ_r+equ_dDG:.2f}]'''

    ############################################################
    # now NEQ
    #tmp = np.append(r.W_mm_to_qml, r.W_qml_to_mm * -1)
    #min_F = abs(min(tmp))
    w_F, w_R = r.W_mm_to_qml, r.W_qml_to_mm
    rexp_f, rexp_f_lower, rexp_f_upper = bootstrap_exp(exp, w_F)
    rexp_r, rexp_r_lower, rexp_r_upper = bootstrap_exp(exp, w_R)
    bar_bi, bar_bi_lower, bar_bi_upper = bootstrap_bar(bar, w_F, w_R)   
    
    # offset everything and change sign
    rexp_f, rexp_f_lower, rexp_f_upper = rexp_f-min_F, rexp_f_lower-min_F, rexp_f_upper-min_F
    rexp_r, rexp_r_lower, rexp_r_upper = (rexp_r * -1)-min_F, (rexp_r_lower * -1)-min_F, (rexp_r_upper * -1)-min_F
    bar_bi, bar_bi_lower, bar_bi_upper = bar_bi-min_F, bar_bi_lower-min_F, bar_bi_upper-min_F
    
    print(f'{np.round(rexp_f, 2)} [{np.round(rexp_f_lower, 2)};{np.round(rexp_f_upper, 2)}],',end =" " )
    print(f'{np.round(rexp_r, 2)} [{np.round(rexp_r_lower, 2)};{np.round(rexp_r_upper, 2)}],',end =" " )
    print(f'{np.round(bar_bi, 2)} [{np.round(bar_bi_lower, 2)};{np.round(bar_bi_upper, 2)}]')

    axs[1].set_title(f'NEQ - {system_name}', fontsize=15)

    ax1 = sns.kdeplot(w_F-min_F, ax=axs[1], fill=True, alpha=.5,color=colors[0])
    y1, x = np.histogram(w_F-min_F, density=True)
    y2, x = np.histogram((w_R*-1)-min_F, density=True)
    y_max = max([y1.max(), y2.max()])
    axs[1].scatter(w_F-min_F, np.random.uniform(low=(y_max/6) *  -1, high=0., size=len(w_F)), s=.4, color=colors[0], alpha=0.8)

    ax2 = sns.kdeplot((w_R*-1)-min_F, ax=axs[1], fill=True, alpha=.5,color=colors[-1])
    axs[1].scatter((w_R*-1)-min_F, np.random.uniform(low=(y_max/6) *  -1, high=0., size=len(w_R)), s=.4, color=colors[-1], alpha=0.8)

    # bidirectional estimate
    axs[1].axvline(x = bar_bi,
            color = 'red',lw=3,ls=':', alpha=.5,
            label = r'$\Delta G_{bid}$')
    # EXP forward
    axs[1].axvline(x = rexp_f, 
            color = colors[0], lw=3,ls=':', alpha=.5,
            label = '$\Delta G_{forw}$')
    # EXP rev
    axs[1].axvline(x = rexp_r, 
            color = colors[-1],lw=3,ls=':', alpha=.5,
            label = '$\Delta G_{rev}$')
    # equ dG
    axs[1].axvline(x=equ_r, 
            color = 'black',lw=3,ls='-', alpha=.5,
            label = r'$\Delta G_{equ}$')


    textstr_neq = f'''$\Delta G_{{forw}}$ = {rexp_f:.2f} [{rexp_f_lower:.2f}; {rexp_f_upper:.2f}] 
$\Delta G_{{rev}}$ = {rexp_r:.2f} [{rexp_r_upper:.2f}; {rexp_r_lower:.2f}]
$\Delta G_{{bid}}$ = {bar_bi:.2f} [{bar_bi_lower:.2f}; {bar_bi_upper:.2f}]'''

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    axs[0].text(0.05, 0.95, textstr_fep, transform=axs[0].transAxes, fontsize=15,
            verticalalignment='top', bbox=props,horizontalalignment='left')
    axs[1].text(0.05, 0.95, textstr_neq, transform=axs[1].transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    
    # set axis labels and legend for both axis
    axs[0].get_yaxis().set_ticks([])
    axs[0].set_ylabel('Density',fontsize=14 )
    axs[1].get_yaxis().set_ticks([])
    axs[1].set_ylabel('Density',fontsize=14)
    axs[0].legend(loc='upper right',fontsize=14, fancybox=True, framealpha=0.5)
        

def plot_only_neq(rs, system_name, axs):
    colors = sns.color_palette("flare", n_colors = 201) 

    # get all dEs
    dEs = []
    for r in rs:
            dEs.extend(r.dE_mm_to_qml)
            dEs.extend(r.dE_qml_to_mm * -1)
    # offset value
    min_F = min(dEs)
    # equ dG
    equ_rs = [equ.compute_free_energy_differences()["Delta_f"][0][-1] for equ in rs[0].equ_mbar]

    # offset everything and change sign
    equ_r = np.average(equ_rs)-min_F
    # 90% CI: https://sphweb.bumc.bu.edu/otlt/MPH-Modules/PH717-QuantCore/PH717-Module6-RandomError/PH717-Module6-RandomError11.html
    # CI = +- t-score(/sigma / sqrt(n))
    equ_dDG = 2.920 * (np.std(equ_rs) / np.sqrt(len(equ_rs))) 

    ############################################################
    # now NEQ
    switching_times = ['5 ps', '10 ps', '20 ps', '50 ps']
    for idx, r in enumerate(rs):
            #tmp = np.append(r.W_mm_to_qml, r.W_qml_to_mm * -1)
            #min_F = abs(min(tmp))
            w_F, w_R = r.W_mm_to_qml, r.W_qml_to_mm
            rexp_f, rexp_f_lower, rexp_f_upper = bootstrap_exp(exp, w_F)
            rexp_r, rexp_r_lower, rexp_r_upper = bootstrap_exp(exp, w_R)

            bar_bi, bar_bi_lower, bar_bi_upper = bootstrap_bar(bar, w_F, w_R)


            # offset everything and change sign
            rexp_f, rexp_f_lower, rexp_f_upper = rexp_f-min_F, rexp_f_lower-min_F, rexp_f_upper-min_F
            rexp_r, rexp_r_lower, rexp_r_upper = (rexp_r * -1)-min_F, (rexp_r_lower * -1)-min_F, (rexp_r_upper * -1)-min_F
            bar_bi, bar_bi_lower, bar_bi_upper = bar_bi-min_F, bar_bi_lower-min_F, bar_bi_upper-min_F

            axs[idx].set_title(f'NEQ - {system_name} - {switching_times[idx]}', fontsize=15)

            ax1 = sns.kdeplot(w_F-min_F, ax=axs[idx], fill=True, alpha=.5,color=colors[0])
            y1, x = np.histogram(w_F-min_F, density=True)
            y2, x = np.histogram((w_R*-1)-min_F, density=True)
            y_max = max([y1.max(), y2.max()])
            axs[idx].scatter(w_F-min_F, np.random.uniform(low=(y_max/6) *  -1, high=0., size=len(w_F)), s=.4, color=colors[0], alpha=0.8)

            ax2 = sns.kdeplot((w_R*-1)-min_F, ax=axs[idx], fill=True, alpha=.5,color=colors[-1])
            axs[idx].scatter((w_R*-1)-min_F, np.random.uniform(low=(y_max/6) *  -1, high=0., size=len(w_R)), s=.4, color=colors[-1], alpha=0.8)

            # bidirectional estimate
            axs[idx].axvline(x = bar_bi,
                    color = 'red',lw=3,ls=':', alpha=.5,
                    label = r'$\Delta G_{bid}$')
            # EXP forward
            axs[idx].axvline(x = rexp_f, 
                    color = colors[0], lw=3,ls=':', alpha=.5,
                    label = '$\Delta G_{forw}$')
            # EXP rev
            axs[idx].axvline(x = rexp_r, 
                    color = colors[-1],lw=3,ls=':', alpha=.5,
                    label = '$\Delta G_{rev}$')
            # equ dG
            axs[idx].axvline(x=equ_r, 
                    color = 'black',lw=3,ls='-', alpha=.5,
                    label = r'$\Delta G_{equ}$')


            textstr_neq = f'''$\Delta G_{{forw}}$ = {rexp_f:.2f} [{rexp_f_lower:.2f}; {rexp_f_upper:.2f}] 
$\Delta G_{{rev}}$ = {rexp_r:.2f} [{rexp_r_upper:.2f}; {rexp_r_lower:.2f}]
$\Delta G_{{bid}}$ = {bar_bi:.2f} [{bar_bi_lower:.2f}; {bar_bi_upper:.2f}]'''
            #$\Delta G_{{equ}}$ = {equ_r:.2f} [{equ_r-equ_dDG:.2f}; {equ_r+equ_dDG:.2f}]'''


            # these are matplotlib.patch.Patch properties
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

            # place a text box in upper left in axes coords
            axs[idx].text(0.05, 0.95, textstr_neq, transform=axs[idx].transAxes, fontsize=15,
                    verticalalignment='top', bbox=props)

            # set axis labels and legend for both axis
            axs[idx].get_yaxis().set_ticks([])
            axs[idx].set_ylabel('Density',fontsize=14 )
            axs[0].legend(loc='upper right',fontsize=14, fancybox=True, framealpha=0.5)
        

def grouper(n, iterable, padvalue=None):
    "grouper(3, 'abcdefg', 'x') --> ('a','b','c'), ('d','e','f'), ('g','x','x')"
    return zip_longest(*[iter(iterable)]*n, fillvalue=padvalue)


all_used_systems = [x for x in zinc_systems if x[0] not in blacklist]
all_systems_results = {}
switching_lengths = [5_000, 10_000, 20_000, 50_000]
for neq_switching_length in switching_lengths:
    all_systems_results[neq_switching_length] = pickle.load(open(f'../data/switching_results/all_results_{neq_switching_length}.pickle', 'rb'))


In [ ]:
# Plot per hipen system the non-equilibrium and instantaneous work distribution 

neq_switching_length = 5_000

for batch_idx, list_of_systems in enumerate(grouper(6, all_used_systems, ('','',''))):
    print(list_of_systems)
    fig, axs = plt.subplots(len(list_of_systems), 2, figsize=(13.0, 17.0), dpi=600)
    for idx, (system_name, smiles, hipen_id) in enumerate(list_of_systems):
        # skip if no system name is provided
        if not system_name:
            continue

        r = all_systems_results[neq_switching_length][hipen_id]
        plot_dist(r, hipen_id, axs[idx])
            
    plt.tight_layout()
    plt.savefig(f'../figures/{batch_idx}_batch_dist_{neq_switching_length=}.pdf')
    plt.show()


In [ ]:
# Plot per hipen system the non-equilibrium for each switching time 

for batch_idx, list_of_systems in enumerate(grouper(6, all_used_systems, ('','',''))):
    print(list_of_systems)
    fig, axs = plt.subplots(len(list_of_systems), 4, figsize=(25.0, 13.0), dpi=600)

    for idx, (system_name, smiles, hipen_id) in enumerate(list_of_systems):
        # skip if no system name is provided
        if not system_name:
            continue
        rs = [ all_systems_results[neq_switching_length][hipen_id] for neq_switching_length in switching_lengths]

        plot_only_neq(rs, hipen_id, axs[idx])
            
    plt.tight_layout()
    plt.savefig(f'../figures/{batch_idx}_batch_neq_dist_for_each_switching_length.pdf')
    plt.show()


In [ ]:
# plot a summary of NEQ and FEP results (offset by the MFES results)
all_systems_results = {}


sns.set_style("ticks")
colors = sns.color_palette("YlOrBr", n_colors = 4) 
ns = [k for k in all_systems_results[neq_switching_length].keys()]

fig, axs = plt.subplots(1, 2, figsize=(8.0, 5.0), dpi=600)

for system in ns:
        equ_rs = [equ.compute_free_energy_differences()["Delta_f"][0][-1] for equ in all_systems_results[5_000][system].equ_mbar]
        equ_r = np.average(equ_rs)

        f_forw = exp(all_systems_results[neq_switching_length][system].dE_mm_to_qml)['Delta_f']
        f_rev = exp(all_systems_results[neq_switching_length][system].dE_qml_to_mm)['Delta_f']
        f_bid = bar(all_systems_results[neq_switching_length][system].dE_mm_to_qml, all_systems_results[neq_switching_length][system].dE_qml_to_mm)['Delta_f']
        
        axs[0].plot(f_forw-equ_r, system, '>', label='forw', c=colors[2], alpha=.8)
        axs[0].plot((f_rev*-1)-equ_r, system,  '<', label='rev', c=colors[2], alpha=.8)
        axs[0].plot((f_bid)-equ_r, system,  'o', label='bid', c = colors[2], alpha=.8)


handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend(handles[:3], labels[:3])
axs[0].axvline(1, linestyle='--' ,color='black')
axs[0].axvline(-1, linestyle='--' ,color='black')
axs[0].axvline(0, linestyle=':' ,color='red')

for system in ns:
    equ_rs = [equ.compute_free_energy_differences()["Delta_f"][0][-1] for equ in all_systems_results[5_000][system].equ_mbar]
    equ_r = np.average(equ_rs)

    for idx, neq_switching_length in enumerate(switching_lengths):
        f_forw = exp(all_systems_results[neq_switching_length][system].W_mm_to_qml)['Delta_f']
        f_rev = exp(all_systems_results[neq_switching_length][system].W_qml_to_mm)['Delta_f']
        f_bid = bar(all_systems_results[neq_switching_length][system].W_mm_to_qml, all_systems_results[neq_switching_length][system].W_qml_to_mm)['Delta_f']

        axs[1].plot(f_forw-equ_r, system,  '>', label='forw', c=colors[idx], alpha=.8)
        axs[1].plot((f_rev*-1)-equ_r, system,  '<', label='rev', c=colors[idx], alpha=.8)
        axs[1].plot((f_bid)-equ_r, system,  'o', label='bid', c = colors[idx], alpha=.8)

axs[1].axvline(1, linestyle='--' ,color='black')
axs[1].axvline(-1, linestyle='--' ,color='black')
axs[1].axvline(0, linestyle=':' ,color='red')
handles, labels = axs[1].get_legend_handles_labels()
axs[1].legend(handles[:9], [f'{int(int(x)/1_000):2} ps: {label:<4}' for label, x in zip(labels[:9], np.repeat(switching_lengths, 3))])

#axs[0].set_xlim((-2,2))
#axs[1].set_xlim((-5,5))

axs[0].set_title('FEP')
axs[1].set_title('NEQ')

axs[0].invert_yaxis()
axs[1].invert_yaxis()

axs[0].set_xlabel(r'$\Delta$G(MFES, FEP) [kT]')
axs[1].set_xlabel(r'$\Delta$G(MFES, NEQ) [kT]')

#axs[1].set_yticklabels(labels=[])
sns.despine()
plt.savefig(f'neq_fep_overview.pdf')
plt.show()


In [ ]:
# plot \sigma(V) for different switching lengths

import seaborn as sns

def return_pattern(num):
    pattern = [0, 1, 2]
    return pattern[num % 3]

def custom_mapping(input_value):
    return ((input_value) // 3) 

ns = [k for k in all_systems_results[neq_switching_length].keys()]

fig, axs = plt.subplots(int(len(ns)/3), 3, figsize=(6.5, 10.0), dpi=600)
for system_idx, system in enumerate(ns):
    f_forw_var = []
    f_rev_var = []
    for idx, neq_switching_length in enumerate(switching_lengths):
        f_forw_var.append(np.std(all_systems_results[neq_switching_length][system].W_mm_to_qml))
        f_rev_var.append(np.std(all_systems_results[neq_switching_length][system].W_qml_to_mm))
    #print(custom_mapping(system_idx), return_pattern(system_idx))
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].plot(f_forw_var, 'o-',label='forw',  alpha=.8)
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].plot(f_rev_var, 'x-',label='rev',  alpha=.8)
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].set_xticks([])
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].set_title(system)
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].set_xticks(np.arange(0, 4, step=1), ['5', '10', '20', '50'])  
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].set_ylabel(r'$\sigma(W)$')
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].set_xlabel('switching length [ps]')
    axs[custom_mapping(system_idx)][return_pattern(system_idx)].legend()


fig.suptitle(r'$\sigma(W)$ as function of switching length')


sns.despine()
plt.tight_layout()
plt.savefig(f'../figures/neq_W_sigma.pdf')
plt.show()


In [ ]:
energy = parse_energy_matrix(open('../data/torsion-scans/hp2_torsion_6_5_7_15_and_7_8_9_16_ani2x//scan-final.xyz'))
axs = sns.heatmap(energy, annot=False, cmap="Spectral_r") # Note: cmap="Spectral_r" is the default

axs.set_ylabel(r'$\chi_{1}$', fontsize=18)
axs.set_xlabel(r'$\chi_{2}$', fontsize=18)

ticks = [i for i in np.linspace(0, 60, 10)]
axs.set_xticks(ticks)
axs.set_yticks(ticks)

interval = [str(int(i)) for i in np.linspace(-180, 180, 10)]
axs.set_xticklabels(interval)
axs.set_yticklabels(interval)
axs.invert_yaxis()
plt.show()
